# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [51]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased.'''

## 2. Ask for parameters

In [91]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [92]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

* **x<sub>A</sub>:**  Number of units purchased from Vendor A (in thousands)
* **x<sub>B1</sub>:** Number of units purchased from Vendor B within the range of 22,000,000-70,000,000 (in thousands)
* **x<sub>B2</sub>:** Number of units purchased from Vendor B within the range of 70,000,001-100,000,000 (in thousands)
* **x<sub>B3</sub>:** Number of units purchased from Vendor B within the range of 100,000,001-150,000,000 (in thousands)
* **x<sub>B4</sub>:** Number of units purchased from Vendor B within the range of 150,000,001-160,000,000 (in thousands)
* **x<sub>C</sub>:** Number of units purchased from Vendor C (in thousands)
* **x<sub>D</sub>:** Number of units purchased from Vendor D (in thousands)
* **x<sub>E1</sub>:** Number of units purchased from Vendor E within the range of 0-42,000,000 (in thousands)
* **x<sub>E2</sub>:** Number of units purchased from Vendor E within the range of 42,000,001-77,000,000 (in thousands) 

**Binary Variables:**

* **y<sub>B1</sub>:** 1 if purchasing from Vendor B within the range of 22,000,000-70,000,000; 0 otherwise.
* **y<sub>B2</sub>:** 1 if purchasing from Vendor B within the range of 70,000,001-100,000,000; 0 otherwise.
* **y<sub>B3</sub>:** 1 if purchasing from Vendor B within the range of 100,000,001-150,000,000; 0 otherwise.
* **y<sub>B4</sub>:** 1 if purchasing from Vendor B within the range of 150,000,001-160,000,000; 0 otherwise.
* **y<sub>E1</sub>:** 1 if purchasing from Vendor E within the range of 0-42,000,000; 0 otherwise.
* **y<sub>E2</sub>:** 1 if purchasing from Vendor E within the range of 42,000,001-77,000,000; 0 otherwise. 


# 2. Ask for objective

In [95]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [96]:
Markdown(response2.text)

Minimize: 

```
3855.34 + 61.150x<sub>A</sub> + 125804.84y<sub>B1</sub> + 68.099x<sub>B1</sub> + 269304.84y<sub>B2</sub> + 66.049x<sub>B2</sub> + 464304.84y<sub>B3</sub> + 64.099x<sub>B3</sub> + 761304.84y<sub>B4</sub> + 62.119x<sub>B4</sub> + 13456.00 + 62.019x<sub>C</sub> + 6583.98 + 72.488x<sub>D</sub> + 70.150x<sub>E1</sub> + 84000y<sub>E2</sub> + Σ<sub>i=1 to 35000</sub> (68.150 - 0.05i)(x<sub>E2</sub> - 42000 - i + 1) 
``` 


# 3. Ask for constraints

In [97]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [98]:
Markdown(response3.text)

## Constraints:

**1. Demand Fulfillment:**

* x<sub>A</sub> + x<sub>B1</sub> + x<sub>B2</sub> + x<sub>B3</sub> + x<sub>B4</sub> + x<sub>C</sub> + x<sub>D</sub> + x<sub>E1</sub> + x<sub>E2</sub> = 239,600.48 

**2. Vendor Capacity:**

* x<sub>A</sub> ≤ 33,000 
* x<sub>B1</sub> + x<sub>B2</sub> + x<sub>B3</sub> + x<sub>B4</sub> ≤ 160,000
* x<sub>C</sub> ≤ 165,600 
* x<sub>D</sub> ≤ 12,000
* x<sub>E1</sub> + x<sub>E2</sub> ≤ 77,000 

**3. Vendor B Quantity Ranges:**

* 22,000y<sub>B1</sub> ≤ x<sub>B1</sub> ≤ 70,000y<sub>B1</sub>
* 70,001y<sub>B2</sub> ≤ x<sub>B2</sub> ≤ 100,000y<sub>B2</sub>
* 100,001y<sub>B3</sub> ≤ x<sub>B3</sub> ≤ 150,000y<sub>B3</sub>
* 150,001y<sub>B4</sub> ≤ x<sub>B4</sub> ≤ 160,000y<sub>B4</sub>
* y<sub>B1</sub> + y<sub>B2</sub> + y<sub>B3</sub> + y<sub>B4</sub>  ≤ 1

**4. Vendor E Quantity Ranges:**

* 0 ≤ x<sub>E1</sub> ≤ 42,000y<sub>E1</sub>
* 42,001y<sub>E2</sub> ≤ x<sub>E2</sub> ≤ 77,000y<sub>E2</sub>
* y<sub>E1</sub> + y<sub>E2</sub>  ≤ 1 

**5. Binary Variable Constraints:**

* y<sub>B1</sub>, y<sub>B2</sub>, y<sub>B3</sub>, y<sub>B4</sub>, y<sub>E1</sub>, y<sub>E2</sub> ∈ {0, 1}

**6. Non-Negativity:**

* x<sub>A</sub>, x<sub>B1</sub>, x<sub>B2</sub>, x<sub>B3</sub>, x<sub>B4</sub>, x<sub>C</sub>, x<sub>D</sub>, x<sub>E1</sub>, x<sub>E2</sub> ≥ 0 


## 3. Generate the pyomo code

In [99]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [100]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You might need to replace these with your actual data)
demand = 239600.48 

# Model
model = pyo.ConcreteModel()

# Variables
model.xA = pyo.Var(within=pyo.NonNegativeReals)
model.xB1 = pyo.Var(within=pyo.NonNegativeReals)
model.xB2 = pyo.Var(within=pyo.NonNegativeReals)
model.xB3 = pyo.Var(within=pyo.NonNegativeReals)
model.xB4 = pyo.Var(within=pyo.NonNegativeReals)
model.xC = pyo.Var(within=pyo.NonNegativeReals)
model.xD = pyo.Var(within=pyo.NonNegativeReals)
model.xE1 = pyo.Var(within=pyo.NonNegativeReals)
model.xE2 = pyo.Var(within=pyo.NonNegativeReals)

model.yB1 = pyo.Var(within=pyo.Binary)
model.yB2 = pyo.Var(within=pyo.Binary)
model.yB3 = pyo.Var(within=pyo.Binary)
model.yB4 = pyo.Var(within=pyo.Binary)
model.yE1 = pyo.Var(within=pyo.Binary)
model.yE2 = pyo.Var(within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return (3855.34 + 61.150*model.xA + 125804.84*model.yB1 + 68.099*model.xB1 + 
            269304.84*model.yB2 + 66.049*model.xB2 + 464304.84*model.yB3 + 
            64.099*model.xB3 + 761304.84*model.yB4 + 62.119*model.xB4 + 
            13456.00 + 62.019*model.xC + 6583.98 + 72.488*model.xD + 
            70.150*model.xE1 + 84000*model.yE2 + 
            sum((68.150 - 0.05*i)*(model.xE2 - 42000 - i + 1) for i in range(1, 35001)))
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.xA + model.xB1 + model.xB2 + model.xB3 + model.xB4 + model.xC + model.xD + model.xE1 + model.xE2 == demand)

model.vendor_capacity_A = pyo.Constraint(expr=model.xA <= 33000)
model.vendor_capacity_B = pyo.Constraint(expr=model.xB1 + model.xB2 + model.xB3 + model.xB4 <= 160000)
model.vendor_capacity_C = pyo.Constraint(expr=model.xC <= 165600)
model.vendor_capacity_D = pyo.Constraint(expr=model.xD <= 12000)
model.vendor_capacity_E = pyo.Constraint(expr=model.xE1 + model.xE2 <= 77000)

model.vendor_B_range_1 = pyo.Constraint(expr=22000*model.yB1 <= model.xB1)
model.vendor_B_range_2 = pyo.Constraint(expr=model.xB1 <= 70000*model.yB1)
model.vendor_B_range_3 = pyo.Constraint(expr=70001*model.yB2 <= model.xB2)
model.vendor_B_range_4 = pyo.Constraint(expr=model.xB2 <= 100000*model.yB2)
model.vendor_B_range_5 = pyo.Constraint(expr=100001*model.yB3 <= model.xB3)
model.vendor_B_range_6 = pyo.Constraint(expr=model.xB3 <= 150000*model.yB3)
model.vendor_B_range_7 = pyo.Constraint(expr=150001*model.yB4 <= model.xB4)
model.vendor_B_range_8 = pyo.Constraint(expr=model.xB4 <= 160000*model.yB4)
model.vendor_B_selection = pyo.Constraint(expr=model.yB1 + model.yB2 + model.yB3 + model.yB4 <= 1)

model.vendor_E_range_1 = pyo.Constraint(expr=0 <= model.xE1)
model.vendor_E_range_2 = pyo.Constraint(expr=model.xE1 <= 42000*model.yE1)
model.vendor_E_range_3 = pyo.Constraint(expr=42001*model.yE2 <= model.xE2)
model.vendor_E_range_4 = pyo.Constraint(expr=model.xE2 <= 77000*model.yE2)
model.vendor_E_selection = pyo.Constraint(expr=model.yE1 + model.yE2 <= 1)

# Solve
solver = pyo.SolverFactory('Your Solver Name')  # Replace 'Your Solver Name' 
results = solver.solve(model)

# Print results
print(results)
print("Objective Value:", pyo.value(model.obj)) 

# Access variable values
print("xA:", pyo.value(model.xA))
print("xB1:", pyo.value(model.xB1))
# ... and so on for other variables
```

**Explanation:**

1. **Import Pyomo:**  Begin by importing the Pyomo library.
2. **Create a Model:** Create a ConcreteModel to represent your optimization problem.
3. **Define Variables:** Define all decision variables with their respective domains (e.g., NonNegativeReals for quantities, Binary for binary variables).
4. **Objective Function:**  Define a function to calculate the total cost and set it as the objective to be minimized.
5. **Add Constraints:** Add each constraint one by one using clear expressions.  
6. **Solve the Model:** Select a suitable solver (e.g., 'gurobi', 'cplex', 'glpk') and solve the model.
7. **Print Results:** Print the solver status and access the optimal values of the decision variables.

**Important Notes:**

* **Replace "Your Solver Name"**: with the actual name of the solver you have installed (e.g., 'gurobi', 'cplex', 'glpk', etc.). Make sure the solver is installed and properly configured in your environment.
* **Sample Data:**  The code uses `demand = 239600.48` as sample data. You should replace this and any other sample data with your real data.
* **Large Summation:** The summation in the objective function is unrolled for clarity. In practice, you can often express such terms more efficiently within Pyomo, potentially using sets and indexed variables.

This code provides a framework for solving your optimization problem in Pyomo. Remember to adapt the code with your specific solver and data, and to interpret the results to make informed decisions. 


## 4. Input problem data and try running the generated code

In [101]:
import pyomo.environ as pyo

# Sample Data (You might need to replace these with your actual data)
demand = 239600.48 

# Model
model = pyo.ConcreteModel()

# Variables
model.xA = pyo.Var(within=pyo.NonNegativeReals)
model.xB1 = pyo.Var(within=pyo.NonNegativeReals)
model.xB2 = pyo.Var(within=pyo.NonNegativeReals)
model.xB3 = pyo.Var(within=pyo.NonNegativeReals)
model.xB4 = pyo.Var(within=pyo.NonNegativeReals)
model.xC = pyo.Var(within=pyo.NonNegativeReals)
model.xD = pyo.Var(within=pyo.NonNegativeReals)
model.xE1 = pyo.Var(within=pyo.NonNegativeReals)
model.xE2 = pyo.Var(within=pyo.NonNegativeReals)

model.yB1 = pyo.Var(within=pyo.Binary)
model.yB2 = pyo.Var(within=pyo.Binary)
model.yB3 = pyo.Var(within=pyo.Binary)
model.yB4 = pyo.Var(within=pyo.Binary)
model.yE1 = pyo.Var(within=pyo.Binary)
model.yE2 = pyo.Var(within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return (3855.34 + 61.150*model.xA + 125804.84*model.yB1 + 68.099*model.xB1 + 
            269304.84*model.yB2 + 66.049*model.xB2 + 464304.84*model.yB3 + 
            64.099*model.xB3 + 761304.84*model.yB4 + 62.119*model.xB4 + 
            13456.00 + 62.019*model.xC + 6583.98 + 72.488*model.xD + 
            70.150*model.xE1 + 84000*model.yE2 + 
            sum((68.150 - 0.05*i)*(model.xE2 - 42000 - i + 1) for i in range(1, 35001)))
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.xA + model.xB1 + model.xB2 + model.xB3 + model.xB4 + model.xC + model.xD + model.xE1 + model.xE2 == demand)

model.vendor_capacity_A = pyo.Constraint(expr=model.xA <= 33000)
model.vendor_capacity_B = pyo.Constraint(expr=model.xB1 + model.xB2 + model.xB3 + model.xB4 <= 160000)
model.vendor_capacity_C = pyo.Constraint(expr=model.xC <= 165600)
model.vendor_capacity_D = pyo.Constraint(expr=model.xD <= 12000)
model.vendor_capacity_E = pyo.Constraint(expr=model.xE1 + model.xE2 <= 77000)

model.vendor_B_range_1 = pyo.Constraint(expr=22000*model.yB1 <= model.xB1)
model.vendor_B_range_2 = pyo.Constraint(expr=model.xB1 <= 70000*model.yB1)
model.vendor_B_range_3 = pyo.Constraint(expr=70001*model.yB2 <= model.xB2)
model.vendor_B_range_4 = pyo.Constraint(expr=model.xB2 <= 100000*model.yB2)
model.vendor_B_range_5 = pyo.Constraint(expr=100001*model.yB3 <= model.xB3)
model.vendor_B_range_6 = pyo.Constraint(expr=model.xB3 <= 150000*model.yB3)
model.vendor_B_range_7 = pyo.Constraint(expr=150001*model.yB4 <= model.xB4)
model.vendor_B_range_8 = pyo.Constraint(expr=model.xB4 <= 160000*model.yB4)
model.vendor_B_selection = pyo.Constraint(expr=model.yB1 + model.yB2 + model.yB3 + model.yB4 <= 1)

model.vendor_E_range_1 = pyo.Constraint(expr=0 <= model.xE1)
model.vendor_E_range_2 = pyo.Constraint(expr=model.xE1 <= 42000*model.yE1)
model.vendor_E_range_3 = pyo.Constraint(expr=42001*model.yE2 <= model.xE2)
model.vendor_E_range_4 = pyo.Constraint(expr=model.xE2 <= 77000*model.yE2)
model.vendor_E_selection = pyo.Constraint(expr=model.yE1 + model.yE2 <= 1)

# Solve
solver = pyo.SolverFactory('glpk')  # Replace 'Your Solver Name' 
results = solver.solve(model)

# Print results
print(results)
print("Objective Value:", pyo.value(model.obj)) 

# Access variable values
print("xA:", pyo.value(model.xA))
print("xB1:", pyo.value(model.xB1))
# ... and so on for other variables


Problem: 
- Name: unknown
  Lower bound: -315569061087.786
  Upper bound: -315569061087.786
  Number of objectives: 1
  Number of constraints: 20
  Number of variables: 16
  Number of nonzeros: 47
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.03869056701660156
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective Value: -315569061087.4331
xA: 33000.0
xB1: 0.0


In [102]:
model.pprint()

15 Var Declarations
    xA : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 33000.0 :  None : False : False : NonNegativeReals
    xB1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    xB2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    xB3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    xB4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    xC : Size=1, Index=None
        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
        None :     0 : 129600.48 :  None : False : False : NonNegati

## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [103]:
print(response.text)

## Variables:

* **x<sub>A</sub>:**  Number of units purchased from Vendor A (in thousands)
* **x<sub>B1</sub>:** Number of units purchased from Vendor B within the range of 22,000,000-70,000,000 (in thousands)
* **x<sub>B2</sub>:** Number of units purchased from Vendor B within the range of 70,000,001-100,000,000 (in thousands)
* **x<sub>B3</sub>:** Number of units purchased from Vendor B within the range of 100,000,001-150,000,000 (in thousands)
* **x<sub>B4</sub>:** Number of units purchased from Vendor B within the range of 150,000,001-160,000,000 (in thousands)
* **x<sub>C</sub>:** Number of units purchased from Vendor C (in thousands)
* **x<sub>D</sub>:** Number of units purchased from Vendor D (in thousands)
* **x<sub>E1</sub>:** Number of units purchased from Vendor E within the range of 0-42,000,000 (in thousands)
* **x<sub>E2</sub>:** Number of units purchased from Vendor E within the range of 42,000,001-77,000,000 (in thousands) 

**Binary Variables:**

* **y<sub>B1</sub>:**

In [104]:
print(response2.text)

Minimize: 

```
3855.34 + 61.150x<sub>A</sub> + 125804.84y<sub>B1</sub> + 68.099x<sub>B1</sub> + 269304.84y<sub>B2</sub> + 66.049x<sub>B2</sub> + 464304.84y<sub>B3</sub> + 64.099x<sub>B3</sub> + 761304.84y<sub>B4</sub> + 62.119x<sub>B4</sub> + 13456.00 + 62.019x<sub>C</sub> + 6583.98 + 72.488x<sub>D</sub> + 70.150x<sub>E1</sub> + 84000y<sub>E2</sub> + Σ<sub>i=1 to 35000</sub> (68.150 - 0.05i)(x<sub>E2</sub> - 42000 - i + 1) 
``` 



In [105]:
print(response3.text)

## Constraints:

**1. Demand Fulfillment:**

* x<sub>A</sub> + x<sub>B1</sub> + x<sub>B2</sub> + x<sub>B3</sub> + x<sub>B4</sub> + x<sub>C</sub> + x<sub>D</sub> + x<sub>E1</sub> + x<sub>E2</sub> = 239,600.48 

**2. Vendor Capacity:**

* x<sub>A</sub> ≤ 33,000 
* x<sub>B1</sub> + x<sub>B2</sub> + x<sub>B3</sub> + x<sub>B4</sub> ≤ 160,000
* x<sub>C</sub> ≤ 165,600 
* x<sub>D</sub> ≤ 12,000
* x<sub>E1</sub> + x<sub>E2</sub> ≤ 77,000 

**3. Vendor B Quantity Ranges:**

* 22,000y<sub>B1</sub> ≤ x<sub>B1</sub> ≤ 70,000y<sub>B1</sub>
* 70,001y<sub>B2</sub> ≤ x<sub>B2</sub> ≤ 100,000y<sub>B2</sub>
* 100,001y<sub>B3</sub> ≤ x<sub>B3</sub> ≤ 150,000y<sub>B3</sub>
* 150,001y<sub>B4</sub> ≤ x<sub>B4</sub> ≤ 160,000y<sub>B4</sub>
* y<sub>B1</sub> + y<sub>B2</sub> + y<sub>B3</sub> + y<sub>B4</sub>  ≤ 1

**4. Vendor E Quantity Ranges:**

* 0 ≤ x<sub>E1</sub> ≤ 42,000y<sub>E1</sub>
* 42,001y<sub>E2</sub> ≤ x<sub>E2</sub> ≤ 77,000y<sub>E2</sub>
* y<sub>E1</sub> + y<sub>E2</sub>  ≤ 1 

**5. Binary Varia

In [106]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (You might need to replace these with your actual data)
demand = 239600.48 

# Model
model = pyo.ConcreteModel()

# Variables
model.xA = pyo.Var(within=pyo.NonNegativeReals)
model.xB1 = pyo.Var(within=pyo.NonNegativeReals)
model.xB2 = pyo.Var(within=pyo.NonNegativeReals)
model.xB3 = pyo.Var(within=pyo.NonNegativeReals)
model.xB4 = pyo.Var(within=pyo.NonNegativeReals)
model.xC = pyo.Var(within=pyo.NonNegativeReals)
model.xD = pyo.Var(within=pyo.NonNegativeReals)
model.xE1 = pyo.Var(within=pyo.NonNegativeReals)
model.xE2 = pyo.Var(within=pyo.NonNegativeReals)

model.yB1 = pyo.Var(within=pyo.Binary)
model.yB2 = pyo.Var(within=pyo.Binary)
model.yB3 = pyo.Var(within=pyo.Binary)
model.yB4 = pyo.Var(within=pyo.Binary)
model.yE1 = pyo.Var(within=pyo.Binary)
model.yE2 = pyo.Var(within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return (3855.34 + 61.150*model.xA + 125804.84*model.yB1 + 68.099*model.xB1 + 
            26930